In [22]:
import re
import numpy as np
import shutil
import os
econfig = {'K_sv':'3s 3p 4s','Rb_sv':'4s 4p 5s','Cs_sv': '5s 5p 6s',\
           'Cu_pv':'3p 3d 4s','Ag': '4d 5s','Au':'5d 6s',\
           'Sc_sv': '3s 3p 3d 4s','Y_sv':'4s 4p 4d 5s',\
           'Al':'3s sp','Ga_d':'3d 4s 4p','In_d':'4d 5s 5p','Tl_d':'5d 6s 6p',\
           'As':'4s 4p','Sb':'5s 5p','Bi':'6s 6p',\
           'Cl':'3s 3p','Br':'4s 4p','I':'5s 5p'}
def get_pot(potcar):
    pot_list = []
    with open(potcar,'r') as f:
        lines = f.readlines()
    for l in lines:
        m = re.match(r'[ ]*PAW_PBE ',l)
        if m:
            pot_list.append(l.split()[1])
    return pot_list

def get_formula(poscar):
    e_dict = {}
    with open(poscar,'r') as f:
        lines = f.readlines()
    elements = lines[5].split()
    counts = lines[6].split()
    for i in range(len(elements)):
        e_dict[elements[i]] = int(counts[i])
    return e_dict

def get_nbands_helper(econfig):
    orbital = econfig.split()
    nbands = []
    for o in orbital:
        if re.match('\ds', o):
            nbands.append(1)
        if re.match('\dp', o):
            nbands.append(3)
        if re.match('\dd', o):
            nbands.append(5)
    return sum(nbands)

def get_nbands(potcar, poscar):
    pot_list = get_pot(potcar)
    e_dict = get_formula(poscar)
    elements = []
    counts = []
    for pot in pot_list:
        elements.append(pot.split('_')[0])
        counts.append(e_dict[pot.split('_')[0]])
    
    nbands = []
    for i in range(len(pot_list)):
        nbands.append(get_nbands_helper(econfig[pot_list[i]]))
    return sum(np.array(nbands)*np.array(counts))
        
def get_outcar_nbands(outcar):
    with open(outcar,'r') as f:
        lines = f.readlines()
    for l in lines:
        m = re.search(r"NBANDS=[ ]+([\d]+)", l)
        if m:
            return int((m.group(1)))

def write_lobsterin(directory, template, potcar):
    shutil.copyfile(template, os.path.join(directory, 'lobsterin')) 
    pots = get_pot(potcar)
    with open(os.path.join(directory, 'lobsterin'),'a') as f:
        for pot in pots:
            l = 'basisfunctions '+pot.split('_')[0]+' '+econfig[pot] +'\n'
            f.write(l)
            
def write_incar(directory, template, potcar, poscar):
    nbands_min = get_nbands(potcar, poscar)   
    shutil.copyfile(template, os.path.join(directory, 'INCAR'))
    with open(os.path.join(directory, 'INCAR'),'a') as f:
        f.write('NBANDS = ' + str(nbands_min+5) + '\n')

      

In [21]:
potcar = '/Users/yao/Google Drive/data/2116/lobster/test/Cs2Ag1In1Br6-orbitalwise/POTCAR'
poscar = '/Users/yao/Google Drive/data/2116/lobster/test/Cs2Ag1In1Br6-orbitalwise/POSCAR'
outcar = '/Users/yao/Google Drive/data/2116/lobster/test/Cs2Ag1In1Br6-orbitalwise/OUTCAR'
template = '/Users/yao/Google Drive/data/2116/lobster/test/lobsterin_temp'
template_incar = '/Users/yao/Google Drive/data/2116/lobster/test/INCAR_temp'
directory = '/Users/yao/Google Drive/data/2116/lobster/test/'
write_lobsterin(directory, template, potcar)
write_incar(directory, template_incar, potcar, poscar)

Good: 51 >= 49


False